In [31]:
import os
import re
import random

MACHINE = "gpub-container_g1"
CHECKPOINT = "checkpoint-19.pth"

EVAL_TEXT = """#!/bin/bash
#$-S /bin/bash
#$-cwd
#$-ac d=none
#$-j y
#$-o $HOME/log/{filename}
#$-jc {machine}.24h

# For internet connection
export MY_PROXY_URL="http://10.1.10.1:8080/"
export HTTP_PROXY=$MY_PROXY_URL
export HTTPS_PROXY=$MY_PROXY_URL
export FTP_PROXY=$MY_PROXY_URL
export http_proxy=$MY_PROXY_URL
export https_proxy=$MY_PROXY_URL
export ftp_proxy=$MY_PROXY_URL
export JAVA_HOME=$HOME/.jre/jdk-11.0.21+9-jre
export PATH=$PATH:$JAVA_HOME/bin

source ~/anaconda3/etc/profile.d/conda.sh
conda activate lavin-torch2.1

torchrun --nproc_per_node 1 --master_port {port} eval_dhpr.py \\
    --adapter_path ./outputs/{expname}/{checkpoint} \\
    --batch_size 2
"""


def generate_random_port(start=1024, end=49151):
    return random.randint(start, end)


def get_name(filename="checkpoint-12.pth"):
    # return ckpt12
    # Use regular expression to extract numeric part
    match = re.search(r'\d+', filename)
    if match:
        numeric_part = match.group()
        new_filename = "ckpt" + numeric_part
        return new_filename
    else:
        print("No numeric part found in the filename.")
        return filename

In [32]:
text = """
scripts/dhpr/g16/llama-version1/exp1_var_g16_bs16.sh
scripts/dhpr/g16/llama-version1/exp1_var_g16_bs32.sh
"""

scripts = [l for l in text.split('\n') if l.strip() != '']

eval_dir = "scripts/dhpr/eval"

job_id = 1

for script in scripts:
    script = script.strip()
    expname = os.path.basename(script)
    new_filename = get_name(CHECKPOINT) + "_" + os.path.basename(script)
    new_filepath = os.path.join(eval_dir, new_filename)

    with open(new_filepath, 'w') as f:
        f.write(
            EVAL_TEXT.format(machine=MACHINE,
                             checkpoint=CHECKPOINT,
                             filename=new_filename.split('.')[0],
                             expname=expname.split('.')[0],
                             port=generate_random_port()))

    # print("qsub -N job{} {}".format(job_id, script))
    # print("qsub {filepath}".format(filepath=filepath))
    # print("qsub -hold_jid job{i} {filepath}".format(i=job_id, filepath=new_filepath))
    print("qsub -N eval{i} {filepath}".format(i=job_id, filepath=new_filepath))
    # print("")
    job_id += 1

qsub -N eval1 scripts/dhpr/eval/ckpt19_exp1_var_g16_bs16.sh
qsub -N eval2 scripts/dhpr/eval/ckpt19_exp1_var_g16_bs32.sh
